In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import LSTM_Model
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import save
from pytorch_lightning.callbacks import EarlyStopping

torch.manual_seed(1)

c:\Users\Aksel\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Aksel\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
c:\Users\Aksel\AppData\Local\Programs\Python\Python39\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


Dataset version: 0.8


In [6]:
seed_everything(42)
device = 'cpu'
early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=5, verbose=True, mode='min')
model = LSTM_Model().to(device)
trainer = Trainer(max_epochs=100, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False, callbacks=[early_stop_callback])
trainer.tune(model)

trainer.fit(model)
save(model.state_dict(), 'Location of our saved model')

Global seed set to 42
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Aksel\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Missing logger folder: c:\Users\Aksel\Desktop\BCM_activity_classification\lightning_logs

  | Name   | Type    | Params
-----------------------------------
0 | lstm   | LSTM    | 42.0 K
1 | fc     | Linear  | 645   
2 | output | Sigmoid | 0     
-----------------------------------
42.6 K    Trainable params
0         Non-trainable params
42.6 K    Total params
0.171     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Aksel\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:225: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
